Open the mask file --> slice area of interest.

Using scipy.convolve, check neighbours of mask file (in 8 directions) --> output coastal cells

## Importing packages

In [2]:
from dask.distributed import LocalCluster, Client
import dask.array
import datetime
from datetime import date 
from datetime import datetime
import glob
import numpy as np
import pandas as pd
from scipy import signal
import scipy.ndimage as ndimage
from scipy.ndimage import convolve
from scipy.ndimage.measurements import label, find_objects
import xarray as xr

## Opening the mask file

In [3]:
# NE coast of Tasmania

ds = xr.open_dataset('/g/data/e14/cp3790/Charuni/static_era5.nc').sel(time=slice('1982', '2018'), longitude=slice(147.0, 149.0), latitude=slice(-40.0, -42.0))
mask = xr.where(ds.lsm==0, 0, 1)

In [4]:
mask

<xarray.DataArray 'lsm' (time: 1, latitude: 9, longitude: 9)>
array([[[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]]])
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 2000-01-01

k is an array of weights, with the same number of dimensions as input

In [5]:
k =([[[1, 1, 1],
    [1, 0, 1],
    [1, 1, 1]]])

input array 'mask' is convolved with 'k', mode='constant' which ensures that the input is extended by filling all values beyond the edge with the same constant value

In [6]:
c = convolve(mask, k, mode='constant')

In [7]:
c

array([[[0, 1, 2, 4, 5, 5, 3, 2, 0],
        [0, 1, 4, 6, 8, 8, 5, 3, 0],
        [2, 3, 5, 6, 8, 8, 5, 3, 0],
        [3, 5, 6, 7, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [3, 5, 5, 5, 5, 5, 3, 2, 0]]])

coastal array is 1 when the mask==1 and the sum of neighbouring cells is < 8 (which means at least one cell is neighbouring a 0 (or sea grid cell)

In [8]:
coastal = np.logical_and(mask == 1, c < 8)

In [9]:
coastal.astype(int)

<xarray.DataArray 'lsm' (time: 1, latitude: 9, longitude: 9)>
array([[[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]]])
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 2000-01-01

## Anomaly datasets

In [10]:
# surface air temperature anomalies
tasa = xr.open_dataset('/g/data/e14/cp3790/Charuni/surface_air_temp_anomaly.nc').sel(time=slice('1982', '2018'), longitude=slice(147.0, 149.0), latitude=slice(-40.0, -42.0))

In [11]:
tasa

<xarray.Dataset>
Dimensions:    (latitude: 9, longitude: 9, time: 13514)
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
Data variables:
    anomaly    (time, latitude, longitude) float32 ...

In [12]:
def fix(ds):
    # This function makes the original data array repeatable so that it matches with the shape of the other data array.

    trial = ds
    i = 0
    while i < 13514:  # <-- Here, 13514 corresponds to the timesteps in land dataset 
        trial = xr.concat([trial, ds], 'time')
        i+=1
    trial = trial.isel(time = slice(0,13514))
    # to specify the dates/time 
    trial.coords['time'] = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1) 
    
    t = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1)
    dates = [date.fromordinal(tt.astype(int)) for tt in t]
    
    trial.coords['time'] = dates

    return trial

In [13]:
anom = tasa.anomaly
anom

<xarray.DataArray 'anomaly' (time: 13514, latitude: 9, longitude: 9)>
[1094634 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31

In [14]:
new_coastal = fix(coastal)  # new_coastal will now have the same shape as anom 

In [15]:
new_coastal

<xarray.DataArray 'lsm' (time: 13514, latitude: 9, longitude: 9)>
array([[[False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False, False],
        [ True, False, False, ...,  True, False, False],
        [ True,  True,  True, ...,  True, False, False]],

       [[False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False, False],
        [ True, False, False, ...,  True, False, False],
        [ True,  True,  True, ...,  True, False, False]],

       [[False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False, False],
        [ True, False, False, ...,  True, False, False],
        [ True,  True,  True, ...,  True, False, False]],

       ...,

       [[False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False, False],
        [ True, False, False, ...,  True, False, False],
        [ True,  True,  True, ...,  True, False, False]],

       [[False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False, False],
        [ True, False, False, ...,  True, False, False],
        [ True,  True,  True, ...,  True, False, False]],

       [[False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False, False],
        [ True, False, False, ...,  True, False, False],
        [ True,  True,  True, ...,  True, False, False]]])
Coordinates:
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * time       (time) object 1982-01-01 1982-01-02 ... 2018-12-30 2018-12-31

In [17]:
coast_sel = xr.where(new_coastal==1, anom, 0)  

In [18]:
coast_sel # coast_sel is a new array which shows temperature anomalies in the coastal cells and 0's in the other cells 

<xarray.DataArray (time: 13514, latitude: 9, longitude: 9)>
array([[[ 0.0000000e+00,  0.0000000e+00, -1.5062218e+00, ...,
         -1.3821659e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.3291950e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.3963661e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [-5.7722988e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -4.5522690e-01,  0.0000000e+00,  0.0000000e+00],
        [-5.6428242e+00,  0.0000000e+00,  0.0000000e+00, ...,
          7.4996948e-03,  0.0000000e+00,  0.0000000e+00],
        [-5.2316570e+00, -5.3280468e+00, -5.6140995e+00, ...,
          2.4584007e-01,  0.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00, -1.5429764e+00, ...,
         -1.5287037e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.4059010e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.3597450e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [-6.0802097e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -6.4494324e-01,  0.0000000e+00,  0.0000000e+00],
        [-6.2250595e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -9.6602631e-01,  0.0000000e+00,  0.0000000e+00],
        [-5.9848919e+00, -6.1238995e+00, -6.4121933e+00, ...,
         -1.0132236e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00, -2.0495129e+00, ...,
         -1.8984337e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.8161297e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.8499737e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [-7.5873184e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -7.4194527e-01,  0.0000000e+00,  0.0000000e+00],
        [-8.0051517e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -8.1653214e-01,  0.0000000e+00,  0.0000000e+00],
        [-7.7515488e+00, -7.6008453e+00, -7.6343594e+00, ...,
         -1.1649876e+00,  0.0000000e+00,  0.0000000e+00]],

       ...,

       [[ 0.0000000e+00,  0.0000000e+00,  1.0404243e+00, ...,
          1.9390106e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          1.8872032e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          1.5405483e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [-1.6554661e+00,  0.0000000e+00,  0.0000000e+00, ...,
          2.6181240e+00,  0.0000000e+00,  0.0000000e+00],
        [-1.2328720e+00,  0.0000000e+00,  0.0000000e+00, ...,
          2.3029728e+00,  0.0000000e+00,  0.0000000e+00],
        [-6.7570686e-02, -5.6566429e-01, -8.2858658e-01, ...,
          1.9707794e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00,  1.7784729e+00, ...,
          2.9558735e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          2.9019642e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          2.5954762e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [-1.1056442e+00,  0.0000000e+00,  0.0000000e+00, ...,
          1.5903702e+00,  0.0000000e+00,  0.0000000e+00],
        [ 5.3541183e-02,  0.0000000e+00,  0.0000000e+00, ...,
          1.6307402e+00,  0.0000000e+00,  0.0000000e+00],
        [ 9.2431068e-01, -3.9360619e-01, -1.1728935e+00, ...,
          1.6650734e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00,  1.8265858e+00, ...,
          2.7831211e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          2.8160887e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.

In [41]:
# I want to define a function that will go through each lat,lon pair and when it finds a value, it will store it
# in a new matrix, with time in x axis and lat,lon coordinates in y axis. Then I would use this matrix to find adjacent 
# ocean cells (for each lat,lon pair) using the sst dataset and do a correlation between the resulting  pairs of SST 
# and TAS anomalies.

def lookup_table (da):
    coords = {}
    for i in coast_sel.latitude:
        for j in coast_sel.longitude:
            if coast_sel != 0:
                coords[coast_sel.time] = i,j
    return coords

In [161]:
# sea surface temperature anomalies 
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/MHW-sev/mhw_severity.pc90.*.nc'))
mhw = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1982', '2018')
                                                        
ssta = mhw.ssta.sel(time=slice('1982', '2018'), lon=slice(147.0, 149.0), lat=slice(-42.0, -40.0))